**Import Libraries**

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import warnings
warnings.filterwarnings('ignore')
from scipy.sparse import hstack, csr_matrix

In [3]:
from google.colab import drive #to import google drive data

In [4]:
drive.mount('/content/drive') #to mount the drive data into colab clound and enter the activation code

Mounted at /content/drive


In [5]:
cd /content/drive/"My Drive/Colab Notebooks/Industrial"

/content/drive/My Drive/Colab Notebooks/Industrial


**Importing Database**

In [6]:
train = pd.read_csv('Final_Train_Dataset.csv')
test = pd.read_csv('Final_Test_Dataset.csv')

In [7]:
train.head()

,Unnamed: 0,experience,job_description,job_desig,job_type,key_skills,location,salary,company_name_encoded
0,0,5-7 yrs,Exp: Minimum 5 years;Good understanding of IOC...,Senior Exploit and Vulnerability Researcher,NaN,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687
1,1,10-17 yrs,He should have handled a team of atleast 5-6 d...,Head SCM,NaN,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458
2,2,5-9 yrs,Must be an effective communicator (written & s...,Deputy Manager - Talent Management & Leadershi...,Analytics,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR,15to25,4195
3,3,7-10 yrs,7 - 10 years of overall experience in data e...,Associate Manager Data Engineering,Analytics,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru,10to15,313
4,4,1-3 yrs,Chartered Accountancy degree or MBA in Finance...,TS- GSA- Senior Analyst,NaN,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305


In [8]:
test.head()

,experience,job_description,job_desig,job_type,key_skills,location,company_name_encoded
0,7-12 yrs,Professional experience in Java/J2EE based ser...,IT Technology Senior Consultant/java/ J2ee/ Se...,NaN,"Java, J2Ee, Tomcat, JBoss, Weblogic, Oracle, E...",Bengaluru,2066
1,0-5 yrs,We are looking for 20+ Fresher/Experienced Can...,Medical Billing Process | International KPO | ...,NaN,"Medical Billing, Insurance Processing",Ahmedabad(Sola),2629
2,3-6 yrs,Should understand overall integration framewor...,Oracle Sales Cloud Functional Consultant,NaN,"Oracle Sales, Functional Consultancy, Troubles...",Bengaluru,2448
3,0-3 yrs,NaN,Looking For Freshers WHO WANT To Work WITH US,NaN,"offline, online, internet, part time, home bas...","Delhi NCR, Chennai, Hyderabad, Gurgaon, Luckno...",2711
4,0-5 yrs,NaN,Process Associate / Sr Process Associate / Tec...,NaN,"voice support, analytical skills, Process asso...",Hyderabad,40


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19802 entries, 0 to 19801
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            19802 non-null  int64 
 1   experience            19802 non-null  object
 2   job_description       15384 non-null  object
 3   job_desig             19802 non-null  object
 4   job_type              4797 non-null   object
 5   key_skills            19801 non-null  object
 6   location              19802 non-null  object
 7   salary                19802 non-null  object
 8   company_name_encoded  19802 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1.4+ MB


**Cleaning Dataset**

In [11]:
train = train.dropna(subset=['key_skills'])
df_train = train[['key_skills', 'job_desig', 'job_description', 'location', 'job_type', 'experience','salary']]
df_test = test[['key_skills', 'job_desig', 'job_description', 'location', 'job_type', 'experience', ]]

In [14]:
def clean_skills(skl):
    skills = str(skl).lower()
    skills = re.sub(r'\...','', skills)
    skills = re.sub(r',','', skills)
    skills = re.sub(r'\s+', ' ', skills)
    return skills

df_train['skills_cleaned'] = df_train['key_skills'].apply(clean_skills)
df_test['skills_cleaned'] = df_test['key_skills'].apply(clean_skills)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
def clean_job_desig(desig):
    job_desig = desig.lower()
    job_desig = re.sub(r'[^a-z]', ' ', job_desig)
    job_desig = re.sub(r'\s+', ' ', job_desig)
    return job_desig

df_train['desig_cleaned'] = df_train['job_desig'].apply(clean_job_desig)
df_test['desig_cleaned'] = df_test['job_desig'].apply(clean_job_desig)

In [16]:
train['job_description'].fillna('missing', inplace=True)
test['job_description'].fillna('missing', inplace=True)

def clean_job_desc(job):
    job_desc = str(job).lower()
    job_desc = re.sub(r'[^a-z]', ' ', job_desc)
    job_desc = re.sub(r'\s+', ' ', job_desc)
    return job_desc

df_train['job_desc_cleaned'] = df_train['job_description'].apply(clean_job_desc)
df_test['job_desc_cleaned'] = df_test['job_description'].apply(clean_job_desc)

In [17]:
def clean_location(loc):
    location = loc.lower()
    location = re.sub(r'[^a-z]', ' ', location)
    location = re.sub(r'\s+', ' ', location)
    return location

df_train['loc_cleaned'] = df_train['location'].apply(clean_location)
df_test['loc_cleaned'] = df_test['location'].apply(clean_location)

In [18]:
train['job_type'].fillna('missingjobtype', inplace=True)
train['job_type'].replace('Analytics', 'analytics', inplace=True)
train['job_type'].replace('Analytic', 'analytics', inplace=True)
train['job_type'].replace('ANALYTICS', 'analytics', inplace=True)
train['job_type'].replace('analytic', 'analytics', inplace=True)

test['job_type'].fillna('missingjobtype', inplace=True)
test['job_type'].replace('Analytics', 'analytics', inplace=True)
test['job_type'].replace('Analytic', 'analytics', inplace=True)
test['job_type'].replace('ANALYTICS', 'analytics', inplace=True)
test['job_type'].replace('analytic', 'analytics', inplace=True)

df_train['job_type_cleaned'] = train['job_type'] 
df_test['job_type_cleaned'] = test['job_type']

In [19]:
def min_exp(exp):
    val = re.sub(r'\-',' ', exp)
    val = val.split(' ')
    val = int(val[0])
    return val

def max_exp(exp):
    val = re.sub(r'\-',' ', exp)
    val = val.split(' ')
    val = int(val[1])
    return val

df_train['min_exp'] = df_train['experience'].apply(min_exp)
df_train['max_exp'] = df_train['experience'].apply(max_exp)

df_test['min_exp'] = df_test['experience'].apply(min_exp)
df_test['max_exp'] = df_test['experience'].apply(max_exp)

In [21]:
df_train['merged'] = (df_train['desig_cleaned'] + ' ' + df_train['job_desc_cleaned'] + ' ' + df_train['skills_cleaned']
                      + ' ' + df_train['job_type_cleaned'])

df_test['merged'] = (df_test['desig_cleaned'] + ' ' + df_test['job_desc_cleaned'] + ' ' + df_test['skills_cleaned']
                     + ' ' + df_test['job_type_cleaned'])

In [22]:
df_train.head()

,key_skills,job_desig,job_description,location,job_type,experience,salary,skills_cleaned,desig_cleaned,job_desc_cleaned,loc_cleaned,job_type_cleaned,min_exp,max_exp,merged
0,"team skills, communication skills, analytical ...",Senior Exploit and Vulnerability Researcher,Exp: Minimum 5 years;Good understanding of IOC...,Delhi NCR(Vikas Puri),NaN,5-7 yrs,6to10,team skills communication skills analytical sk...,senior exploit and vulnerability researcher,exp minimum years good understanding of ioc ru...,delhi ncr vikas puri,missingjobtype,5,7,senior exploit and vulnerability researcher ex...
1,"ppc, logistics, inventory management, supply c...",Head SCM,He should have handled a team of atleast 5-6 d...,Sonepat,NaN,10-17 yrs,10to15,ppc logistics inventory management supply chai...,head scm,he should have handled a team of atleast direc...,sonepat,missingjobtype,10,17,head scm he should have handled a team of atle...
2,"HR Analytics, Employee Engagement, Training, S...",Deputy Manager - Talent Management & Leadershi...,Must be an effective communicator (written & s...,Delhi NCR,Analytics,5-9 yrs,15to25,hr analytics employee engagement training succ...,deputy manager talent management leadership de...,must be an effective communicator written spok...,delhi ncr,analytics,5,9,deputy manager talent management leadership de...
3,"SQL, Javascript, Automation, Python, Ruby, Ana...",Associate Manager Data Engineering,7 - 10 years of overall experience in data e...,Bengaluru,Analytics,7-10 yrs,10to15,sql javascript automation python ruby analytic...,associate manager data engineering,years of overall experience in data engineeri...,bengaluru,analytics,7,10,associate manager data engineering years of o...
4,"accounting, finance, cash flow, financial plan...",TS- GSA- Senior Analyst,Chartered Accountancy degree or MBA in Finance...,Gurgaon,NaN,1-3 yrs,3to6,accounting finance cash flow financial plannin...,ts gsa senior analyst,chartered accountancy degree or mba in finance...,gurgaon,missingjobtype,1,3,ts gsa senior analyst chartered accountancy de...


In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train['salary'] = le.fit_transform(df_train['salary'])

In [24]:
df_train.head()

,key_skills,job_desig,job_description,location,job_type,experience,salary,skills_cleaned,desig_cleaned,job_desc_cleaned,loc_cleaned,job_type_cleaned,min_exp,max_exp,merged
0,"team skills, communication skills, analytical ...",Senior Exploit and Vulnerability Researcher,Exp: Minimum 5 years;Good understanding of IOC...,Delhi NCR(Vikas Puri),NaN,5-7 yrs,5,team skills communication skills analytical sk...,senior exploit and vulnerability researcher,exp minimum years good understanding of ioc ru...,delhi ncr vikas puri,missingjobtype,5,7,senior exploit and vulnerability researcher ex...
1,"ppc, logistics, inventory management, supply c...",Head SCM,He should have handled a team of atleast 5-6 d...,Sonepat,NaN,10-17 yrs,1,ppc logistics inventory management supply chai...,head scm,he should have handled a team of atleast direc...,sonepat,missingjobtype,10,17,head scm he should have handled a team of atle...
2,"HR Analytics, Employee Engagement, Training, S...",Deputy Manager - Talent Management & Leadershi...,Must be an effective communicator (written & s...,Delhi NCR,Analytics,5-9 yrs,2,hr analytics employee engagement training succ...,deputy manager talent management leadership de...,must be an effective communicator written spok...,delhi ncr,analytics,5,9,deputy manager talent management leadership de...
3,"SQL, Javascript, Automation, Python, Ruby, Ana...",Associate Manager Data Engineering,7 - 10 years of overall experience in data e...,Bengaluru,Analytics,7-10 yrs,1,sql javascript automation python ruby analytic...,associate manager data engineering,years of overall experience in data engineeri...,bengaluru,analytics,7,10,associate manager data engineering years of o...
4,"accounting, finance, cash flow, financial plan...",TS- GSA- Senior Analyst,Chartered Accountancy degree or MBA in Finance...,Gurgaon,NaN,1-3 yrs,4,accounting finance cash flow financial plannin...,ts gsa senior analyst,chartered accountancy degree or mba in finance...,gurgaon,missingjobtype,1,3,ts gsa senior analyst chartered accountancy de...


In [25]:
df_test.head()

,key_skills,job_desig,job_description,location,job_type,experience,skills_cleaned,desig_cleaned,job_desc_cleaned,loc_cleaned,job_type_cleaned,min_exp,max_exp,merged
0,"Java, J2Ee, Tomcat, JBoss, Weblogic, Oracle, E...",IT Technology Senior Consultant/java/ J2ee/ Se...,Professional experience in Java/J2EE based ser...,Bengaluru,NaN,7-12 yrs,java j2ee tomcat jboss weblogic oracle elastic...,it technology senior consultant java j ee seve...,professional experience in java j ee based ser...,bengaluru,missingjobtype,7,12,it technology senior consultant java j ee seve...
1,"Medical Billing, Insurance Processing",Medical Billing Process | International KPO | ...,We are looking for 20+ Fresher/Experienced Can...,Ahmedabad(Sola),NaN,0-5 yrs,medical billing insurance processing,medical billing process international kpo insu...,we are looking for fresher experienced candida...,ahmedabad sola,missingjobtype,0,5,medical billing process international kpo insu...
2,"Oracle Sales, Functional Consultancy, Troubles...",Oracle Sales Cloud Functional Consultant,Should understand overall integration framewor...,Bengaluru,NaN,3-6 yrs,oracle sales functional consultancy troublesho...,oracle sales cloud functional consultant,should understand overall integration framewor...,bengaluru,missingjobtype,3,6,oracle sales cloud functional consultant shoul...
3,"offline, online, internet, part time, home bas...",Looking For Freshers WHO WANT To Work WITH US,NaN,"Delhi NCR, Chennai, Hyderabad, Gurgaon, Luckno...",NaN,0-3 yrs,offline online internet part time home base wo...,looking for freshers who want to work with us,nan,delhi ncr chennai hyderabad gurgaon lucknow mu...,missingjobtype,0,3,looking for freshers who want to work with us ...
4,"voice support, analytical skills, Process asso...",Process Associate / Sr Process Associate / Tec...,NaN,Hyderabad,NaN,0-5 yrs,voice support analytical skills process associ...,process associate sr process associate tech as...,nan,hyderabad,missingjobtype,0,5,process associate sr process associate tech as...


In [53]:
X=df_train[['min_exp','max_exp']]
Y=df_train['salary']

In [54]:
X.shape

(19801, 2)

In [55]:
Y.shape

(19801,)

**Train Test Split**

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(X,Y, test_size=0.20,random_state=42)

In [58]:
print(x_train.shape)
print(x_cv.shape)
print(y_train.shape)
print(y_cv.shape)

(15840, 2)
(3961, 2)
(15840,)
(3961,)


**Building the Model**

In [59]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [63]:
Y_pred=clf.predict(x_cv)

**Accuracy**

In [64]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(Y_pred,y_cv)
acc

0.4072203988891694

**Final Prediction**

In [65]:
X_test_=df_test[['min_exp','max_exp']]

In [66]:
final_pred=clf.predict(X_test_)

In [67]:
y_pred_class = le.inverse_transform(final_pred)

In [69]:
y_pred_class

array(['15to25', '0to3', '6to10', ..., '15to25', '15to25', '15to25'],
      dtype=object)

In [70]:
df_sub = pd.DataFrame(data=y_pred_class, columns=['salary'])

In [73]:
df_sub.to_csv( "output.csv", index=False, encoding='utf-8-sig')